In [1]:
# Small LLM / Notebook created by Javier Ideami (ideami.com)
# Typical LLMs need many GPUs and millions of dollars to be trained
# This code trains a small LLM with a single GPU and little GPU memory 
# Of course results are not like a chatGPT, but they are good enough to see how the LLM trains to go
# from random combinations of letters to actual words and phrases that are sometimes decently coherent
# GPT3 has 175 Billion parameters. GPT4 has many, many more.
# This model has only 19 Million parameters with its default settings. That's why its perfect for learning 
# and experimenting

# Official notebook #vj30

In [2]:
#### For GOOGLE COLAB and similar platform Users:
#### Make sure to select a GPU in the online platform. Don't run this code with a CPU (it will be too slow)

# If you are running this code locally, your GPU should be selected automatically

In [3]:
# uncomment and run the following installation lines ONLY if you havent installed these libraries already outside of the notebook
#!pip install ipdb -q
#!pip install tqdm -q
#!pip install sentencepiece -q
#!pip install wandb -q

# And if you are not in Google Colab and you didn't yet install Pytorch, make sure to do it:
# find the ideal pytorch installation command at https://pytorch.org/get-started/locally/

In [4]:
### Import necessary libraries

import os, sys
import ipdb # for debugging
from tqdm import tqdm
from datetime import datetime
import platform, shutil # detect platform type
import requests, zipfile, io 

# Pytorch
import torch
import torch.nn as nn
from torch.nn import functional as F

import sentencepiece as spm # For the tokenizer

# These lines improve performance for Ampere Architecture (e.g: A100s)
# torch.backends.cuda.matmul.allow_tf32 = True  # allow tf32 on matmul
# torch.backends.cudnn.allow_tf32 = True  # allow tf32 on cudnn
# Empty GPU cache memory
# torch.cuda.empty_cache()

if torch.backends.mps.is_available():
    device = "mps"
else:
    print("MPS device not found.")

In [5]:
# Download necessary files and create necessary folders
# wiki.txt - dataset: a tiny segment of the English Wikipedia
# wiki_tokenizer.model: trained tokenizer file (in another notebook I show you how to produce this file)
# wiki_tokenizer.vocab: trained tokenizer file (in another notebook I show you how to produce this file)
# encoded_data.pt (dataset tokenized with the tokenizer)
# I will explain how to produce encoded_data.pt - because it takes quite a bit to process, it's nice to have it in advance

# NOTE: Downloading will take a while, be patient. You can refresh your folder from time to time to see when the files
# have been created. If you have any problems downloading the files with this code, I have also added llm_train.zip
# to the downloadable resources of this lecture (however, best option is to use this code, because then you don't need
# to upload the files or do anything else)

files_url = "https://ideami.com/llm_train"

# Downloading proceeds if we detect that one of the key files to download is not present
if not os.path.exists(f"encoded_data.pt"):
    print("Downloading files using Python")
    response = requests.get(files_url)
    zipfile.ZipFile(io.BytesIO(response.content)).extractall(".")
else:
    print("you seem to have already downloaded the files. If you wish to re-download them, delete the encoded_data.pt file")



In [6]:
# Set main parameters

# ARCHITECTURE PARAMETERS
batch_size= 8 # How many samples do we train at once (set as needed, typical range 8 to 128)
              # 8 is good for a GPU with 4GB of memory, 128 is good for a GPU with 24GB of memory
context=512 # Sequence length used for training, 512 is a good compromise for our level of resources
embed_size=384 # Embedding size
n_layers = 7 # Number of transformer layers
n_heads = 7 # Number of heads within each layer
BIAS = True # Do we want Bias parameters?

# HYPERPARAMETERS
lr = 3e-4 # Initial learning rate
dropout=0.05 # Dropout percentage
weight_decay = 0.01 # Weight decay regularizer
grad_clip = 1.0 # Gradient clipping to prevent gradient explosion

# TRAINING parameters
train_iters = 15000 # Maximum number of training iterations
eval_interval=50 # How often do we evaluate the performance?
eval_iters=3 # Number of iterations while we evaluate performance
compile = False # Compile will accelerate performance in compatible systems
load_pretrained = True # Do we want to load a pretrained model to continue training?

checkpoint_dir = 'models/'  # Where do we store checkpoints?

checkpoint_fn = "latest.pt" 
# Name of checkpoint file to be saved during training

checkpoint_load_fn = "latest.pt" 
# Name of checkpoint file to be loaded when load_pretrained is True
# You can load llm2.pt to experiment with a checkpoint that already reached 2.31 of loss

dtype = torch.bfloat16 # our target internal data type

# MODE
# Do we want to run the model in inference mode?
inference=False 

# DEVICE - Sets device to GPU or CPU (use GPU always)
device = "cuda" if torch.cuda.is_available() else "mps"
print("device: You will be using: ",device)


device: You will be using:  mps


In [7]:
# LOGGING parameters
# When you run this cell, it will ask you to enter your Wandb API Key, which you
# can find at your account on https://wandb.ai/settings#api
wandb_log = True
wandb_project = "llm_udemy"
wandb_run_name = "basic-gpt" + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

if wandb_log:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name)

# The first time you run this logging code set to True, the weights and biases library
# will ask you for an API key. You can follow the instructions in the video, or you can
# also simply click on a link that should appear when you run this cell, pointing to this
# address: https://wandb.ai/authorize  
# Going to that address will allow you to quickly get an API key as well


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: meinczinger (meinczinger-personal-use). Use `wandb login --relogin` to force relogin


In [8]:
with open('../data/wiki.txt', 'r', encoding='utf-8') as f:
    text=f.read()

print(text[10000:10500])

FileNotFoundError: [Errno 2] No such file or directory: '../data/wiki.txt'

In [9]:
# SENTENCEPIECE TOKENIZER

# Load trained tokenizer
# Make sure that " model_file = " is pointing to the right file
sp = spm.SentencePieceProcessor(model_file='wiki_tokenizer.model')

# Get the vocabulary size of our tokenizer
vocab_size = sp.get_piece_size()
print(f"Tokenizer vocab_size: {vocab_size}")

# Create the encoding and decoding tokenizer functions
encode = lambda s: sp.Encode(s)
decode = lambda l: sp.Decode(l)

# Test that encoding and decoding are working well
print(decode(encode("Encoding Decoding functions ready")))
print(encode("Some sample text"))

Tokenizer vocab_size: 4096
Encoding Decoding functions ready
[1149, 271, 555, 296, 3927]


In [10]:
# Tokenization of the dataset
if os.path.exists(f"encoded_data.pt"):
    # Load encoded data if you already saved it previously
    print("Loading saved encoded data")
    data = torch.load('encoded_data.pt')
else:
    # If you still didn't encode and save the encoding, do it here
    print("Encoding data")
    data = torch.tensor(encode(text), dtype=torch.long)
    torch.save(data, 'encoded_data.pt')


Loading saved encoded data


/var/folders/x1/5s7srkw11qn8zndqtnz1w43r0000gn/T/ipykernel_3120/795341420.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('encoded_data.pt')


In [11]:
data_size=len(data) # Get the size of the dataset

spl = int(0.9*data_size) # set the split at 90%-10%
train_data=data[:spl] # training data will be 90% of the dataset
val_data=data[spl:] # validation data will be 10% of the dataset
print(f'Total data: {data_size/1e6:.2f} Million | Training: {len(train_data)/1e6:.2f} Million | Validation: {len(val_data)/1e6:.2f} Million')

# data[:30] : shows the first 30 token IDs

Total data: 59.21 Million | Training: 53.29 Million | Validation: 5.92 Million


In [12]:
############## HELPER FUNCTIONS ###########################

# Return a batch of either training or evaluation data
def get_batch(split):
    # BS = Batch Size / SL = Sequence Length or context length
    data = train_data if split=="train" else val_data # Select the split
    inds = torch.randint(len(data)-context, (batch_size,)) # (BS)
    x = torch.stack([data[i: i+context] for i in inds]) # (BS,SL)
    y = torch.stack([data[i+1: i+context+1] for i in inds]) # (BS,SL)

    # Examples of what it returns
    # # First 10 elements of first batch of inputs and labels
    #x[0][:10] -> tensor([ 664,  278, 4031, 4056, 4065, 4062, 4062, 4051, 13, 13])
    #y[0][:10] -> tensor([ 278, 4031, 4056, 4065, 4062, 4062, 4051,   13, 13, 4066])

    x,y = x.to(device), y.to(device)
    return x,y



In [13]:
#################################################################################
# Main Training Process
#################################################################################

# Main Setup
from llm_models import ModelFactory
from llm_models.tools import ModelParameters

params_path = "../config/model_params.yaml"

params = ModelParameters(params_path)
model = ModelFactory("basic_gpt", params, vocab_size)

# model = GPT() # Instantiate LLM
model = model.to(dtype) # Set the precision type
model = model.to(device) # Move it to the right device

# Torch.compile compiles a PyTorch model to an optimized version, aiming to improve runtime performance and efficiency.
# Disable if your system doesn't support it
if compile:
    print("Torch :: Compiling model")
    model = torch.compile(model)


# Print the number of parameters of our model (19 million in our case)
print(sum(p.numel() for p in model.parameters()) / 1e6, " Million parameters")

19.837954  Million parameters


In [14]:
# Calculate the Loss
@torch.no_grad()  # Prevent gradient calculation
def calculate_loss():
    out={}
    model.eval()
    for split in ['train','eval']:        
        l=torch.zeros(eval_iters)  # Create a tensor of zeros the size of eval_iters
        for i in range(eval_iters):
            x,y=get_batch(split) # Get a new batch of data
            _,loss=model(x,y)  # Calculate the loss
            l[i]=loss  # Store the loss in the next position of tensor
        out[split]=l.mean().item()  # Calculate the mean and extract the final value
    model.train()
    return out

l=calculate_loss()
print(l)

{'train': 8.395833015441895, 'eval': 8.395833015441895}


In [15]:
# Generate a new sample
@torch.no_grad()
def generate_sample(input):
    t1 = torch.tensor(encode(input), dtype=torch.long, device=device) # Tokenize string -> (tensor of ids)
    t1 = t1[None,:]  # (1 , [size of ids])
    newgen = model.generate(t1,max=64)[0].tolist() # call the generate method, limit output size
    result=decode(newgen) # decode the result with the tokenizer to get back characters
    print(f"{result}")

generate_sample("The mountain in my city is") # Generate a sample

The mountain in my city is engine Eictous Green st fl her Syza calellow Pre repl eightemroywrit Bor��ets Ihi joinitectwar7soci goodgroundisionswoodened studio Musature Scott ethessj thingsFran territory Hon Conisements stud village List Mod energy potrse peritiz story hor road Pre Los Rep


In [16]:
#################################################################################
# Main Training Process
#################################################################################

# Set Weight Decay differently for different kinds of parameters
# parameter dictionary where keys are parameter names, and values are the parameter themselves
p_dict = {p_name: p for p_name, p in model.named_parameters() if p.requires_grad} # len: 370

# isolate weight matrices as they benefit specially from weight decay
weight_decay_p = [p for n, p in p_dict.items() if p.dim() >= 2]  # len: 171

# isolate other parameters like bias parameters, that don't benefit from weight decay
no_weight_decay_p = [p for n, p in p_dict.items() if p.dim() < 2] # len: 199

# store the parameter types in a list of dictionaries
optimizer_groups = [
    {'params': weight_decay_p, 'weight_decay': weight_decay},
    {'params': no_weight_decay_p, 'weight_decay': 0.0}
]

# Declare optimizer, it helps us compute gradients, update parameters, manage learning rate, apply weight decay
optimizer = torch.optim.AdamW(optimizer_groups, lr=lr, betas=(0.9, 0.99))
# betas: control the exponential moving averages of the gradient and its square,
# which are essential components of the Adam and AdamW optimization algorithms.

# Declare scheduler to change learning rate through the training
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_iters, eta_min=lr/10)
# learning rate will descend till a minimum of a tenth of the lr

start_iteration = 0
best_val_loss = float('inf')  # Track best loss value


In [17]:
# Loading Checkpoints

# Loads a previously saved checkpoint
def load_checkpoint(path):
    print("LLM - Loading model")
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict']) # Load parameters
    optimizer.load_state_dict(checkpoint['optimizer_state_dict']) # Load optimizer state
    iteration = checkpoint['iteration'] # In what iteration did we save the model?
    loss = checkpoint['loss'] # What was the last loss value?
    print(f"Loaded iter {iteration} with loss {loss}")
    return iteration, loss

################# OPTIONAL : LOAD A PREVIOUS CHECKPOINT
if os.path.exists(f"{checkpoint_dir}/{checkpoint_load_fn}") and load_pretrained:
    start_iteration, loss = load_checkpoint(checkpoint_dir + checkpoint_load_fn)
    best_val_loss = loss

LLM - Loading model


/var/folders/x1/5s7srkw11qn8zndqtnz1w43r0000gn/T/ipykernel_3120/2082066746.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


Loaded iter 9600 with loss 2.84375


In [18]:
#### INFERENCE MODE - Activate inference and then exit
if inference==True:
    model.eval()
    while True:
         qs = input("Enter text (q to quit) >>> ")
         if qs == "":
             continue
         if qs == 'q':
             break
         generate_sample(qs)

In [19]:
#################################################################
###################### TRAINING #################################
#################################################################

try:
    for i in tqdm(range(start_iteration, train_iters)):
        xb,yb = get_batch("train") # Get a new batch of data
        logits,loss = model(xb,yb) # Run the LLM and get the logits and the loss

        if (i % eval_interval==0 or i == train_iters-1): # Calculate the loss
            l = calculate_loss()
            print(f"\n{i}: train loss: {l['train']} / val loss: {l['eval']}")

            # We do a quick test so that we observe the evolution through the training
            # Remember that we use a very small dataset which doesn't include all topics
            generate_sample("The mountain in my city is") # Generate a sample

            if l['eval'] < best_val_loss: # If we improved the best loss, save a checkpoint
                best_val_loss = l['eval']
                print("[CHECKPOINT]: Saving with loss: ", best_val_loss)
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': best_val_loss,
                    'iteration': i,
                }, checkpoint_dir + checkpoint_fn)

        if wandb_log:
            wandb.log({
                    "loss/train": l['train'],
                    "loss/val": l['eval'],
                    "lr": scheduler.get_last_lr()[0],
                },
                step = i)

        optimizer.zero_grad(set_to_none=True) # Reset gradients
        loss.backward() # Calculate new gradients

        # This line clips the gradients to prevent the exploding gradient problem during training.
        # Exploding gradients can occur when gradients become too large, causing unstable updates to model weights.
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=grad_clip)

        optimizer.step() # Update the model parameters
        scheduler.step() # Update the learning rate value

    if wandb_log:
        wandb.finish()


except KeyboardInterrupt:
    print("Training interrupted. Cleaning up...")

finally:
    # Release GPU memory
    torch.mps.empty_cache()
    print("GPU memory released.")

if wandb_log:   
    wandb.finish()
torch.mps.empty_cache()

# Code designed by Javier ideami
# ideami.com


  0%|          | 0/5400 [00:00<?, ?it/s]


9600: train loss: 3.1197917461395264 / val loss: 2.9114582538604736
The mountain in my city is an urban area in 80. It was named during the row of .

Neo, in the end of the, cantons, Saint John (the state of Alpine – Arpine in 1929) was the first people to start the name


  1%|          | 50/5400 [01:26<2:01:02,  1.36s/it]


9650: train loss: 2.96875 / val loss: 3.0833332538604736
The mountain in my city is a three-through the world dog's Joe Philippines.*Other professor Robert Perth hospital were opened to about 45 days in Muso, the Joe Bell. Then Flames should be included at Joe La roford Palace, 880 intended to the


  2%|▏         | 100/5400 [02:47<1:58:04,  1.34s/it]

...

100%|█████████▉| 5399/5400 [2:41:33<00:00,  1.21it/s]


14999: train loss: 3.0677082538604736 / val loss: 2.9739582538604736
The mountain in my city is actively bebered to are the behaviourflies of molar goresprignouns. Therefore allowing it the costume of the lake. Since its mountains changes ray, far and oil pathillars do not be in about half of the city. Sent


100%|██████████| 5400/5400 [2:41:38<00:00,  1.80s/it]


loss/train,▇▆▂███▆▆▇▆▇▅▅▄▄▅▅▅▅▅▄▄▄▃▅▅▄▄▂▆▂▄▃▅▁▆▆▆▇▆
loss/val,▅▅▄▆▅█▃▄▄▇▄▄▇▄▅▃█▆▅▄▆▄▃▄▄▄▁▅▅▂▃▅▄▄▅▂▄▄▂▂
lr,███████████████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▃▃▃▂▂▁
loss/train,3.06771
loss/val,2.97396
lr,0.00012


GPU memory released.
